# Generative Adversarial Network

**Objective**:
Build a Generative Adversarial network for MNIST images.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import sys

## Define GAN

In [3]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

Using TensorFlow backend.


## Load the Dataset

In [0]:
 (X_train, X_test), (Y_train, Y_test) = mnist.load_data()

In [0]:
# Note that for GAN, we are going to use only the train images. No labels are required.
# Our objective of the discriminator is to find out whether the image is fake or real.
#X_train[0]

# rescale the images : -1 to 1 instead of 0 to 255

X_train = X_train / 127.5 -1

In [0]:
#X_train[0]

In [20]:
X_train.shape

(60000, 28, 28)

In [0]:
?np.expand_dims

In [0]:
X_train = np.expand_dims(X_train, axis=3)

In [22]:
X_train.shape

(60000, 28, 28, 1)

## Configurations

In [0]:
# MNIST image details
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

## Latent Space (vectors) dimension
latent_dim = 100

# optimizer
optimizer = Adam(0.0002, 0.5)


## Utility Functions

### Discriminator

In [0]:
#
# Fully connected network
##  inputs: image -> output: fake/real status
def build_discriminator():
  
  model = Sequential()
  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  # output is fake / real
  model.add(Dense(1, activation='sigmoid'))

  model.summary()

  input = Input(shape=img_shape)
  out = model(input)
  # use Model Functional API
  return Model(inputs=input, outputs=out)


### Generator

In [0]:
## generator 
#
# Fully conected network
#
##  inputs: latent space -> output: image
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  # create the image
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()

  # input is noise
  input = Input(shape=(latent_dim,))
  out = model(input)

  return Model(inputs=input, outputs=out)

## Build GAN

In [10]:
# build and compile discriminator
# 
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                            optimizer=optimizer,
                            metrics=['accuracy'])

W0801 11:24:31.697007 139920668424064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0801 11:24:31.711242 139920668424064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0801 11:24:31.781128 139920668424064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0801 11:24:31.788537 139920668424064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0801 11:24:31.794745 139920668424064 deprecation.py:323] From /usr/local/lib/python3.6

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [16]:
## build generator
generator = build_generator()

W0801 11:30:46.263050 139920668424064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
__________

### Combined Model - GAN

In [0]:
# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
validity = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [18]:
combined.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_2 (Model)              (None, 28, 28, 1)         1493520   
_________________________________________________________________
model_1 (Model)              (None, 1)                 533505    
Total params: 2,027,025
Trainable params: 1,489,936
Non-trainable params: 537,089
_________________________________________________________________


## Training GAN

In [0]:
epochs = 30000
batch_size = 32
sample_interval = 200

In [0]:
!mkdir /content/images1

In [0]:
#get the samples
# As training was done, check the generated images at regular intervals.
#
def sample_images(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
      axs[i,j].axis('off')
      cnt += 1
  fig.savefig("/content/images1/%d.png" % epoch)
  plt.close()

In [32]:
# build Adversarial ground truths for discriminator and generator
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Train both the models.
#
for epoch in range(epochs):

  #--------------------
  # train discriminator
  #--------------------

  # Select a random batch of images
  idx = np.random.randint(0, X_train.shape[0], batch_size)
  imgs = X_train[idx]

  # Generate a batch of new images, using generator
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  gen_imgs = generator.predict(noise)

  # train generator using both real and fake images.
  d_loss_real = discriminator.train_on_batch(imgs, valid)
  d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  #-------------
  # train generator
  #-----------------
  noise = np.random.normal(0, 1, (batch_size, latent_dim))

  # Train the generator (to have the discriminator label samples as valid)
  g_loss = combined.train_on_batch(noise, valid)



  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
    # Plot the progress
    print ("Epoch %d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
    sample_images(epoch)
        

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 0 [D loss: 0.643345, acc.: 67.19%] [G loss: 0.861467]
Epoch 200 [D loss: 0.608127, acc.: 71.88%] [G loss: 0.771584]
Epoch 400 [D loss: 0.586186, acc.: 79.69%] [G loss: 0.942852]
Epoch 600 [D loss: 0.600886, acc.: 64.06%] [G loss: 0.915870]
Epoch 800 [D loss: 0.523256, acc.: 81.25%] [G loss: 1.014366]
Epoch 1000 [D loss: 0.506490, acc.: 82.81%] [G loss: 1.032790]
Epoch 1200 [D loss: 0.591494, acc.: 64.06%] [G loss: 0.933244]
Epoch 1400 [D loss: 0.604301, acc.: 64.06%] [G loss: 0.959686]
Epoch 1600 [D loss: 0.628410, acc.: 71.88%] [G loss: 1.012214]
Epoch 1800 [D loss: 0.609851, acc.: 70.31%] [G loss: 0.984196]
Epoch 2000 [D loss: 0.591792, acc.: 65.62%] [G loss: 0.963633]
Epoch 2200 [D loss: 0.550544, acc.: 73.44%] [G loss: 1.015582]
Epoch 2400 [D loss: 0.560318, acc.: 75.00%] [G loss: 0.980694]
Epoch 2600 [D loss: 0.553365, acc.: 75.00%] [G loss: 1.000778]
Epoch 2800 [D loss: 0.558361, acc.: 75.00%] [G loss: 1.035558]
Epoch 3000 [D loss: 0.607478, acc.: 70.31%] [G loss: 0.991809]